# Upload big datasets to HuggingFace with unstable connection

Script made by GPT but working.

In [ ]:
from huggingface_hub import login
login("") # replace with your token

from huggingface_hub import HfApi, list_repo_files
from pathlib import Path
import os

# 🔧 CONFIGURATION
REPO_ID = "Hibou-Foundation/big_ds_preprocessed_spectogram_1"
LOCAL_DIR = "/mnt/drive/big_ds_1.hf"
REPO_TYPE = "dataset"

# ⚙️ Initialize the API
api = HfApi()

def upload_directory(local_dir: str, repo_id: str, repo_type: str = "dataset"):
    """
    Uploads all files from `local_dir` to the Hugging Face Hub repository.
    Automatically skips already uploaded files (resumable).
    """
    local_dir = Path(local_dir)
    if not local_dir.exists():
        raise ValueError(f"Local directory does not exist: {local_dir}")

    print(f"📂 Scanning directory: {local_dir}")
    files_to_upload = [p for p in local_dir.rglob("*") if p.is_file()]
    print(f"Found {len(files_to_upload)} files to check.")

    # 🧠 Get list of already uploaded files
    print(f"🔍 Fetching existing files in repo: {repo_id}")
    existing_files = set(list_repo_files(repo_id=repo_id, repo_type=repo_type))
    print(f"Repo already has {len(existing_files)} files.")

    uploaded_count = 0
    skipped_count = 0

    for fpath in files_to_upload:
        # Normalize path in repo (relative to base directory)
        path_in_repo = str(fpath.relative_to(local_dir)).replace("\\", "/")

        if path_in_repo in existing_files:
            print(f"⏩ Skipping already uploaded: {path_in_repo}")
            skipped_count += 1
            continue

        try:
            print(f"⬆️ Uploading: {path_in_repo} ...")
            api.upload_file(
                path_or_fileobj=str(fpath),
                path_in_repo=path_in_repo,
                repo_id=repo_id,
                repo_type=repo_type,
            )
            uploaded_count += 1
            print(f"✅ Uploaded: {path_in_repo}")
        except Exception as e:
            print(f"❌ Error uploading {fpath}: {e}")
            print("Stopping — rerun this script to resume.")
            break

    print("\n✅ Upload complete.")
    print(f"Uploaded: {uploaded_count}, Skipped: {skipped_count}")

upload_directory(LOCAL_DIR, REPO_ID, REPO_TYPE)